In [3]:
import numpy as np
import pandas as pd
import tensorflow as tf
import matplotlib.pyplot as plt
from keras.models import Sequential
from keras.layers import Dense
import joblib

np.set_printoptions(suppress=True)
dummydata = pd.read_excel('dummy_data.xlsx')

dummydata.dropna(axis=0, inplace=True)

TargetVariable=['Oil_Yield']
Predictors=['HDPE', 'LDPE', 'PP', 'PS', 'PVC', 'PET', 'Temperature', 'Heating_Rate','Particle_Size','Feed_Size','Catalyst']

x = dummydata[Predictors].values
y = dummydata[TargetVariable].values

from sklearn.preprocessing import StandardScaler
PredictorScaler = StandardScaler()
TargetVarScaler = StandardScaler()

PredictorScalerFit = PredictorScaler.fit(x)
TargetVarScalerFit = TargetVarScaler.fit(y)

x = PredictorScalerFit.transform(x)
y = TargetVarScalerFit.transform(y)

from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.3, random_state=1)

model= Sequential()

# Defining the Input layer and FIRST hidden layer, both are same!
model.add(Dense(units=5, input_dim=X_train.shape[1], kernel_initializer='normal', activation='relu'))
 
# Defining the Second layer of the model
# after the first layer we don't have to specify input_dim as keras configure it automatically
model.add(Dense(units=5, kernel_initializer='normal', activation='relu'))
# The output neuron is a single fully connected node 
# Since we will be predicting a single number
model.add(Dense(1, kernel_initializer='normal'))
 
# Compiling the model
model.compile(loss='mean_squared_error', optimizer='adam')
 
# Fitting the ANN to the Training set
model.fit(X_train, y_train ,batch_size = 20, epochs = 100, verbose=1)

Predictions=model.predict(X_test)
 
# Scaling the predicted Price data back to original
Predictions=TargetVarScalerFit.inverse_transform(Predictions)
 
# Scaling the y_test Price data back to original 
y_test_orig=TargetVarScalerFit.inverse_transform(y_test)
 
# Scaling the test data back to original scale
Test_Data=PredictorScalerFit.inverse_transform(X_test)
 
TestingData=pd.DataFrame(data=Test_Data, columns=Predictors)
TestingData['Oil_Yield']=y_test_orig
TestingData['Predicted_Oil_Yield']=Predictions

APE=100*(abs(TestingData['Oil_Yield']-TestingData['Predicted_Oil_Yield'])/TestingData['Oil_Yield'])
TestingData['APE']=APE

#joblib.dump(PredictorScalerFit, 'x_scaler.joblib')
#joblib.dump(TargetVarScalerFit, 'y_scaler.joblib')
#joblib.dump(model, 'ann_model.joblib')  
print('The Accuracy of ANN model is:', 100-np.mean(APE))

Epoch 1/100
6/6 [==============================] - 1s 2ms/step - loss: 1.0549
Epoch 2/100
6/6 [==============================] - 0s 2ms/step - loss: 1.0541
Epoch 3/100
6/6 [==============================] - 0s 1ms/step - loss: 1.0534
Epoch 4/100
6/6 [==============================] - 0s 2ms/step - loss: 1.0525
Epoch 5/100
6/6 [==============================] - 0s 2ms/step - loss: 1.0516
Epoch 6/100
6/6 [==============================] - 0s 2ms/step - loss: 1.0507
Epoch 7/100
6/6 [==============================] - 0s 2ms/step - loss: 1.0491
Epoch 8/100
6/6 [==============================] - 0s 2ms/step - loss: 1.0471
Epoch 9/100
6/6 [==============================] - 0s 2ms/step - loss: 1.0447
Epoch 10/100
6/6 [==============================] - 0s 2ms/step - loss: 1.0405
Epoch 11/100
6/6 [==============================] - 0s 2ms/step - loss: 1.0358
Epoch 12/100
6/6 [==============================] - 0s 2ms/step - loss: 1.0292
Epoch 13/100
6/6 [==============================] - 0s 2ms/st

In [2]:
TestingData.head()

,HDPE,LDPE,PP,PS,PVC,PET,Temperature,Heating_Rate,Particle_Size,Feed_Size,Catalyst,Oil_Yield,Predicted_Oil_Yield,APE
0,0.0,0.0,0.0,100.0,0.0,-8.881784e-16,450.0,6.0,100.0,60.0,0.0,49.0,57.703247,17.761729
1,70.0,30.0,0.0,0.0,0.0,-8.881784e-16,560.0,10.0,4.0,22.5,1.0,75.0,59.751064,20.331914
2,0.0,0.0,100.0,0.0,0.0,-8.881784e-16,500.0,10.0,5.0,2.0,0.0,81.0,80.236130,0.943050
3,0.0,70.0,0.0,30.0,0.0,-8.881784e-16,450.0,10.0,2.0,10.0,0.0,83.7,75.779282,9.463224
4,0.0,30.0,70.0,0.0,0.0,-8.881784e-16,560.0,10.0,4.0,22.5,1.0,78.0,71.147209,8.785629
